In [1]:
using Pkg
Pkg.activate("..")
using Revise
using Equilibrium
using AD_TAUENN
using AD_TAUENN
using FUSE
using FUSE.IMAS
using Plots; gr();


  Activating environment at `~/.julia/dev/FUSE/Project.toml`


In [2]:
if true
    ϵ = 0.48
    κ = 1.68
    δ = 0.21
    R0 = 6.2
    B0 = 5.3
    ip = 15.E6
    βn = 1.0
    x_point = (5.0, -6.5)
    symmetric=true

    resolution = 129

    dd = IMAS.dd()
    resize!(dd.equilibrium.time_slice,1)
    FUSE.init(dd.equilibrium.time_slice[1]; B0, R0, ϵ, δ, κ, beta_n=βn, ip, x_point=x_point)

    eqactor = FUSE.SolovevEquilibriumActor(dd.equilibrium.time_slice[1], symmetric=symmetric)
    @time FUSE.step(eqactor)
    dd.equilibrium.time_slice[1] = FUSE.finalize(eqactor, resolution, (maximum([R0*(1-ϵ*2),0.0]), R0*(1+ϵ*2)), (-R0*ϵ*κ*2, R0*ϵ*κ*2))
    dd.equilibrium.time_slice[1].global_quantities.li_3 = 1.0 # made up!
    dd.equilibrium.time_slice[1].time = 0.0
    dd.equilibrium.time = [0.0]
    dd.equilibrium.vacuum_toroidal_field.r0 = R0
    dd.equilibrium.vacuum_toroidal_field.b0 = [B0]
    dd.equilibrium.time_slice[1].global_quantities.magnetic_axis.b_field_tor = B0
    dd.equilibrium.time_slice[1].global_quantities.magnetic_axis.r = R0

    plot(dd.equilibrium.time_slice[1], color=:green, psi_levels_out=[])
    #dd.equilibrium.time_slice[1];
elseif false
    filename=joinpath(dirname(abspath(@__FILE__)), "..", "sample","CAT_eq_ods.json")
    dd = IMAS.json2imas(filename);
    @time IMAS.flux_surfaces(dd.equilibrium);
    nothing
else
    filename=joinpath(dirname(abspath(@__FILE__)), "..", "sample","ITER_eq_ods.json")
    dd = IMAS.json2imas(filename);
    @time IMAS.flux_surfaces(dd.equilibrium);
    nothing
end
dd.equilibrium.time_slice[1].boundary.elongation=1.68

 52.680227 seconds (123.20 M allocations: 6.389 GiB, 4.22% gc time, 92.59% compilation time)


1.68

In [15]:
cp.profiles_1d[1]
#@show eq.time_slice[1].boundary.geometric_axis.r, eq.time_slice[1].global_quantities.magnetic_axis.r

electrons
｜density ➡ 101-element Vector{Float64}
｜pressure ➡ Function
｜temperature ➡ 101-element Vector{Float64}
grid
｜rho_tor_norm ➡ 101-element LinRange{Float64}
ion
｜[1]
｜｜density ➡ 101-element Vector{Float64}
｜｜element
｜｜｜[1]
｜｜｜｜a ➡ 2
｜｜｜｜z_n ➡ 1
｜｜label ➡ "D"
｜｜temperature ➡ 101-element Vector{Float64}
｜[2]
｜｜density ➡ 101-element Vector{Float64}
｜｜element
｜｜｜[1]
｜｜｜｜a ➡ 12
｜｜｜｜z_n ➡ 6
｜｜label ➡ "C"
｜｜temperature ➡ 101-element Vector{Float64}
pressure_thermal ➡ 101-element Vector{Float64}
rotation_frequency_tor_sonic ➡ 101-element LinRange{Float64}
zeff ➡ 101-element Vector{Float64}


In [3]:
cp = IMAS.core_profiles()
cs = IMAS.core_sources()
eq = dd.equilibrium
summary = IMAS.summary()

summary.time = [0.0]
resize!(cp.profiles_1d,1)

FUSE.init(cs,eq;Paux_e=1e6,Paux_i=1e6)
FUSE.init(cp.profiles_1d[1], eq.time_slice[1],summary; ne_ped=1e20, ne_peaking=1.5, Te_ped=500, Te_peaking=3, w_ped=0.08, zeff=2, Paux=2e6, n_points=101)
summary

local
｜pedestal
｜｜n_e
｜｜｜value ➡ 1-element Vector{Float64}
｜｜position
｜｜｜rho_tor_norm ➡ 1-element Vector{Float64}
｜｜zeff
｜｜｜value ➡ 1-element Vector{Int64}
time ➡ 1-element Vector{Float64}


In [24]:
resize!(cs.source,2)
cs
resize!(cs.source,5)

[1]
｜identifier
｜｜description ➡ "Arbitrary source from transport initialization"
｜｜index ➡ 901
｜｜name ➡ "arb"
｜profiles_1d
｜｜[1]
｜｜｜electrons
｜｜｜｜power_inside ➡ 129-element Vector{Float64}
｜｜｜grid
｜｜｜｜rho_tor_norm ➡ 129-element Vector{Float64}
｜｜｜｜volume ➡ 129-element Vector{Float64}
｜｜｜total_ion_power_inside ➡ 129-element Vector{Float64}
[2]
[3]
[4]
[5]


In [17]:
eqt = eq.time_slice[1]
tauennactor = FUSE.TaueNNactor(cp,eq, cs, summary)
FUSE.step(tauennactor)

step of tauennactors(eqt.boundary.elongation, ods.summary.local.pedestal.n_e.value[1] / 1.0e19) = (1.68, 10.0)
(wped, pped) = (0.33295206529691085, 0.19689356848960227)
(eqt.boundary.elongation, ods.summary.local.pedestal.n_e.value[1] / 1.0e19) = (1.68, 10.0)
(wped, pped) = (0.33295206529691085, 0.19689356848960227)
(tval[1], tval[end]) = (9549.768315241377, 80.0)


LoadError: DimensionMismatch("arrays could not be broadcast to a common size; got a dimension with lengths 101 and 129")

In [6]:
eqt.global_quantities

beta_normal ➡ 5.9027861254323
beta_pol ➡ 20.34807047299424
beta_tor ➡ 0.009264304952773462
energy_mhd ➡ Function
ip ➡ -2.4755106103230356e6
length_pol ➡ 27.5076792320081
li_3 ➡ 1.0
magnetic_axis
｜b_field_tor ➡ 5.3
｜r ➡ 6.2
｜z ➡ 0.060395007922700486
psi_axis ➡ Function
psi_boundary ➡ Function
q_95 ➡ Function
q_axis ➡ Function


In [7]:
dd2 = IMAS.dd()
dd2.equilibrium = IMAS.equilibrium()
resize!(dd2.equilibrium.time_slice,1)
dd2.equilibrium.time_slice[1] = eqt



boundary
｜elongation ➡ 1.68
｜elongation_lower ➡ Function
｜elongation_upper ➡ Function
｜geometric_axis
｜｜r ➡ Function
｜｜z ➡ Function
｜minor_radius ➡ Function
｜squareness_lower_inner ➡ Function
｜squareness_lower_outer ➡ Function
｜squareness_upper_inner ➡ Function
｜squareness_upper_outer ➡ Function
｜triangularity ➡ Function
｜triangularity_lower ➡ Function
｜triangularity_upper ➡ Function
global_quantities
｜beta_normal ➡ 5.9027861254323
｜beta_pol ➡ 20.34807047299424
｜beta_tor ➡ 0.009264304952773462
｜energy_mhd ➡ Function
｜ip ➡ -2.4755106103230356e6
｜length_pol ➡ 27.5076792320081
｜li_3 ➡ 1.0
｜magnetic_axis
｜｜b_field_tor ➡ 5.3
｜｜r ➡ 6.2
｜｜z ➡ 0.060395007922700486
｜psi_axis ➡ Function
｜psi_boundary ➡ Function
｜q_95 ➡ Function
｜q_axis ➡ Function
profiles_1d
｜dpressure_dpsi ➡ 129-element Vector{Float64}
｜dvolume_dpsi ➡ 129-element Vector{Float64}
｜elongation ➡ 129-element Vector{Float64}
｜f ➡ 129-element Vector{Float64}
｜f_df_dpsi ➡ 129-element Vector{Float64}
｜geometric_axis
｜｜r ➡ Function
｜｜z 

In [8]:
using NNlib
using Flux
using Dates
using AD_EPEDNN
using AD_TGLFNN


# run tauenn from standalone
inputs = TauennInputs()
inputs.q0= 1.2
inputs.mixing = "dc"
inputs.ngrid = 101
inputs.delta = 0.45
inputs.iterations=100
inputs.rho_fluxmatch=0.6
inputs.error=0.001
#display(inputs)
@time outputs = tau_enn(inputs; verbose=true);
display(outputs)
plot(outputs.ods.core_profiles.profiles_1d[1].electrons.temperature, label="$(inputs.Paux)MW")

LoadError: type TauennInputs has no field ne_ped